In [1]:
import cv2
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
import numpy as np

def calculate_laplacian_variance(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

def blur_detect_DIP(image_path):

    image = cv2.imread(image_path)
    threshhold = 100
    laplacian_var = calculate_laplacian_variance(image)
    
    if laplacian_var < threshhold:
        return "blur"
    else:
        return "sharp"

def blur_detect_ML(image_path):
    class FeatureExtractor(nn.Module):
        def __init__(self):
            super(FeatureExtractor, self).__init__()
            self.resnet = models.resnet18(pretrained=True)
            self.resnet.fc = nn.Identity() 
    
        def forward(self, x):
            return self.resnet(x)
    
    #Setting up a shallow Fully Connected Classifier
    class Classifier(nn.Module):
        def __init__(self, input_size, num_classes):
            super(Classifier, self).__init__()
            self.fc1 = nn.Linear(input_size, 128)
            self.fc2 = nn.Linear(128, num_classes)
    
        def forward(self, x):
            x = torch.flatten(x, 1)  # Flatten the features
            x = nn.functional.relu(self.fc1(x))
            x = self.fc2(x)
            return x
    
    feature_extractor = FeatureExtractor()
    classifier = Classifier(512, 3)
    model = nn.Sequential(feature_extractor, classifier)
    
    #Loading our trained model
    model.load_state_dict(torch.load('model.pth'))
    model.eval()
    
    #Defining an inference method for custom images
    def classify_image(image_path, model):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
        image = transform(Image.open(image_path)).unsqueeze(0)
        features = feature_extractor(image)
        outputs = classifier(features)
        _, predicted = torch.max(outputs.data, 1)
        return predicted.item()
    
    #Mapping the class index - To Category
    category = {0: "blur", 1: "blur", 2: "sharp"}
    # image_path = 'img1.jpeg'
    
    class_index = classify_image(image_path, model)
    return category[class_index]

def detect_blur(image_path):
    DIP = blur_detect_DIP(image_path)
    ML = blur_detect_ML(image_path)

    if (DIP == "sharp" and ML == "sharp"):
        return True

    if (DIP == "blur" and ML == "blur"):
        return False

    return False

In [3]:
image_path = '/Users/abdulwadood/Downloads/Archive 2/FYP Final/ErrorChecking/BlurDetection/img1.jpeg'
detect_blur(image_path)

/Users/abdulwadood/anaconda3/envs/InVerse/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/abdulwadood/anaconda3/envs/InVerse/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/abdulwadood/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:03<00:00, 11.7MB/s]


True